In [12]:
import os
import subprocess
import uuid
import asyncio
import yaml
import chromadb
import requests
import base64
import zlib
import json
import argparse
import tempfile
from chromadb.config import Settings
from chromadb.utils import embedding_functions
from autogen_core.memory import MemoryContent, MemoryMimeType
from autogen_core.models import ChatCompletionClient, ModelInfo
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import RoundRobinGroupChat, Swarm, SelectorGroupChat
from autogen_agentchat.conditions import TextMentionTermination, HandoffTermination, TextMentionTermination, MaxMessageTermination
from autogen_agentchat.messages import HandoffMessage
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_ext.memory.chromadb import ChromaDBVectorMemory, PersistentChromaDBVectorMemoryConfig
from autogen_ext.memory.canvas import TextCanvasMemory
from autogen_ext.auth.azure import AzureTokenProvider
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential
from langchain_community.document_loaders import PyMuPDFLoader, TextLoader, UnstructuredWordDocumentLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [13]:
AZURE = "/home/jun/Documents/Master_IA/Code/TFM_BA_GPT/src/model_config_azure.yaml"
OLLAMA = "/home/jun/Documents/Master_IA/Code/TFM_BA_GPT/src/model_config_ollama.yaml"
LMSTUDIO = "C:/Master IA/TFM_BA_GPT/src/model_config_lmstudio.yaml"

# Load model configuration and create the model client.
with open(AZURE, "r") as f:
    model_config = yaml.safe_load(f)
    
model_client = ChatCompletionClient.load_component(model_config)

Agent to make diagram

In [14]:
def read_txt_file(file_path: str) -> str:
    """
    Reads the content of a .txt file and returns it as a string.
    """
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()
    except Exception as e:
        return f"Error reading file: {e}"
    
def read_pdf_file(file_path: str) -> str:
    """
    Reads the content of a .pdf file and returns it as a string.
    """
    try:
        loader = PyMuPDFLoader(file_path)
        documents = loader.load()
        return documents[0].page_content if documents else ""
    except Exception as e:
        return f"Error reading file: {e}"
    
def write_mermaid_to_file(mermaid_code: str, filename: str) -> None:
    """
    Write Mermaid diagram code to a .mmd file.

    Args:
        mermaid_code (str): The Mermaid syntax/code to write.
        filename (str): The name of the file to write to. Should end with '.mmd'.
    """
    if not filename.endswith('.mmd'):
        raise ValueError("Filename must end with '.mmd'")
    
    with open(filename, 'w', encoding='utf-8') as file:
        file.write("```mermaid\n")
        file.write(mermaid_code)
        file.write("\n```")

    print(f"Mermaid diagram written to {filename}")




In [15]:
FILE_NAME = "process_diagram.mmd"

 # --- Set up the Canvas-based Memory ---
text_canvas_memory = TextCanvasMemory()  # Canvas is created internally

    # --- Tools for the Writer to create/update files on the canvas ---
update_file_tool = text_canvas_memory.get_update_file_tool()

In [ ]:
planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
    - summarizer: An agent that uses tools to read pdf files and summarizes the process information then handoff the summary to the diagram_creator_agent.
    - diagram_creator_agent: An agent that receives the summary of a process from the summarizer Agent and creates diagrams as a code using Mermaid languagge then handoff to rmaid_code_reviewer_agent for feedback.
    - mermaid_code_reviewer_agent: An agent that reviews the Mermaid code from the diagram_creator_agent and provides feedback.

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After assigning tasks, wait for all agents to finish their tasks.
    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
    model_client_stream=True,
)

summarizer_agent = AssistantAgent(
        name="summarizer",
        description="An agent that use tools to read pdf files and summarizes the information.",
        model_client=model_client,
        tools=[read_pdf_file],
        system_message=(
            "You are an expert on making process summaries."
            "You will read a pdf file using tools and summarize the information."
            "Make sure to include all the important information."
            "Handoff the summary to the diagram_creator_agent."
            "The summary should be in the format: 'Summary: <summary>'"
        ),
        model_client_stream=True,  # Enable model client streaming.
    )

diagram_creator_agent = AssistantAgent(
    name="diagram_creator_agent",
    description="An agent that create diagrams as a code.",
    model_client=model_client,
    tools=[update_file_tool],
    system_message="""
    You are a Process Diagram Creator and an expert in the Mermaid language.
    Your job is to create process diagrams as a code using the Mermaid language from a process summary.
    You will receive a process summary provided by the summarizer agent and you will generate a diagram in Mermaid language.
    The generated Mermaid code is stored on the canvas in a file named "{FILE_NAME}"
    Save it by calling the 'update_file_tool' tool with the entire Mermaid code
    """,
    model_client_stream=True,
)

mermaid_code_reviewer_agent = AssistantAgent(
        name="mermaid_code_reviewer_agent",
        description="An agent that reviews the Mermaid code.",
        model_client=model_client,
        tools=[write_mermaid_to_file],
        system_message=(
            "You are an expert on the Mermaid languagge."
            "Review the Diagram Mermaid code stored in '{FILE_NAME}' on the canvas."
            "If there is feedback, provide it to the diagram_creator_agent."
            "else use the code with write_mermaid_to_file to save as /home/jun/Documents/Master_IA/Code/TFM_BA_GPT/doc/mermaid.mmd and say 'no feedback'."
        ),
)

In [17]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

In [18]:
selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""

In [19]:
team = SelectorGroupChat(
    [planning_agent, summarizer_agent, diagram_creator_agent, mermaid_code_reviewer_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)

In [20]:
task = "Read the PDF in file_path:'/home/jun/Documents/Master_IA/Code/TFM_BA_GPT/doc/Sales SOP for acquiring new clients.pdf' file and create a diagram as a code for that process using Mermaid language"

In [21]:
await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
Read the PDF in file_path:'/home/jun/Documents/Master_IA/Code/TFM_BA_GPT/doc/Sales SOP for acquiring new clients.pdf' file and create a diagram as a code for that process using Mermaid language
---------- ModelClientStreamingChunkEvent (PlanningAgent) ----------
1. summarizer: Read the PDF file located at '/home/jun/Documents/Master_IA/Code/TFM_BA_GPT/doc/Sales SOP for acquiring new clients.pdf' and summarize the process information for acquiring new clients.

Let me know when the summarizer has completed their task.
---------- ToolCallRequestEvent (summarizer) ----------
[FunctionCall(id='call_WcOfSQEzUKIA6o20U8luJe9b', arguments='{"file_path":"/home/jun/Documents/Master_IA/Code/TFM_BA_GPT/doc/Sales SOP for acquiring new clients.pdf"}', name='read_pdf_file')]
---------- ToolCallExecutionEvent (summarizer) ----------
[FunctionExecutionResult(content="Sales SOP for acquiring new clients \nHere's an example of a standard operating procedure a comp

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content="Read the PDF in file_path:'/home/jun/Documents/Master_IA/Code/TFM_BA_GPT/doc/Sales SOP for acquiring new clients.pdf' file and create a diagram as a code for that process using Mermaid language", type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0), metadata={}, content="1. summarizer: Read the PDF file located at '/home/jun/Documents/Master_IA/Code/TFM_BA_GPT/doc/Sales SOP for acquiring new clients.pdf' and summarize the process information for acquiring new clients.\n\nLet me know when the summarizer has completed their task.", type='TextMessage'), ToolCallRequestEvent(source='summarizer', models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0), metadata={}, content=[FunctionCall(id='call_WcOfSQEzUKIA6o20U8luJe9b', arguments='{"file_path":"/home/jun/Documents/Master_IA/Code/TFM_BA_GPT/doc/Sales SOP for acquiring new clien